###Importing Libraries

In [19]:
# Importing Libraries 
import csv
import pandas as pd
from IPython.display import clear_output
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.preprocessing import StandardScaler, OneHotEncoder, Imputer
from time import time
%pylab inline

Populating the interactive namespace from numpy and matplotlib


### Importing Classifiers

In [20]:
from __future__ import division
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier,AdaBoostClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import StratifiedKFold
from sklearn.cross_validation import train_test_split
from sklearn.svm import LinearSVC
from sklearn import svm
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import mixture
from sklearn import cross_validation


In [21]:
# reading validation files
age=pd.read_csv('id_age_val.csv', index_col='ID')
lab=pd.read_csv('id_time_labs_val.csv')
vit=pd.read_csv('id_time_vitals_val.csv')

In [22]:
# reading test files 
# age=pd.read_csv('C:\Users\HS\Desktop\Xerox\Test_Dataset\id_age_test.csv', index_col='ID')
# vit=pd.read_csv('C:\Users\HS\Desktop\Xerox\Test_Dataset\id_time_vitals_test.csv')
# lab=pd.read_csv('C:\Users\HS\Desktop\Xerox\Test_Dataset\id_time_labs_test.csv')

In [23]:
vit=vit.fillna(vit.median())
vit[[u'V1', u'V2', u'V3', u'V4', u'V5', u'V6']]=vit[[u'V1', u'V2', u'V3', u'V4', u'V5', u'V6']].applymap(lambda x: math.log(1+x))

In [24]:
# filling labs values with ideal values

# values=[7.4,40,85,140,43.5,25,13,0.9,7,48,275,1.1,1400,190, 1.3,0.01,0.01,87.5,87.5, 47.39,4.4,94,25,50,2]
# lab2=lab.copy()
# for i,col in enumerate(lab.columns[2:]):
#     lab[col]=lab[col].fillna(values[i])
#     lab2[col]=values[i] 
    
# result=(lab-lab2)
# result['ID']=lab.ID
# result['TIME']=lab.TIME
# lab=result.copy()
lab=lab.fillna(lab.median())
lab=lab.applymap(lambda x: math.log(1+x))

### Extracting Features

In [25]:
def create_filled (df,df2,i):
#     df_g=df.groupby(by='ID')
    med=df.median()  # median 
    maxm=df.max() 
    minm=df.min()
    max_min=maxm-minm  # maximum - minimum
    
    first_values=df.head(1).values[0][2:8]
    last_values=df.tail(1).values[0][2:8]
    
    t_diff=df.iat[len(df)-1,1]/len(df)

    
    # FEATURES FROM LABS
    # (if separate df is passed for labs
    med_lab=df2.median()  # median 
    maxm_lab=df2.max() 
    minm_lab=df2.min()
    max_min_lab=maxm_lab-minm_lab  # maximum - minimum
    


    #********************* Concatenating final feature vector *********************
#     print np.array([len(df)]).transpose().shape
#     print med.values[1:8]
#     print [t_diff]
#     print (np.array([age.AGE[i]])).shape

    feat=np.concatenate([np.array([age.AGE[i]]),
                         med.values[2:8],
                         maxm.values[2:8], 
                         minm.values[2:8], 
                         max_min.values[2:8],
                         np.array([len(df)]),
                         np.array([t_diff])]
                         , axis=1) 
    
#     lab_features=np.concatenate([med.values[9:34],
#                                  maxm.values[9:34],
#                                  minm.values[9:34],
#                                  max_min.values[9:34]
#                                 ],axis=1)
    
#     print med_lab.values[2:]
    lab_features=np.concatenate([med_lab.values[2:],
                                 maxm_lab.values[2:],
                                 minm_lab.values[2:],
                                 max_min_lab.values[2:]
                                ],axis=1)
    
#     print med.values[9:34].shape
#     print x_submission.shape
#     print lab_features.shape

    x_submission=np.concatenate([feat, 
                                 first_values, 
                                 last_values, 
                                 lab_features
                                 ],axis=1)

    return x_submission


            

###Create x_submission set for all id's

In [26]:
l1,l2=age.index.min(),(age.index.max()+1)
x_submission=[]
stamps=[]
j=0
for i in range(l1,l2):

    lower=j
    print 'ID = ' + str(i)  + '           '+ str(j) 
    df=vit.iloc[lower:j+1]
    df2=lab.iloc[lower:j+1]
#     s=time()
#     print df.iat[(j-lower),0]
    while(int(df.tail(1).ID)==i and j<199971):  

        icu=int(df.tail(1).ICU)
        if icu==1:

            x_sub=create_filled(df,df2,i)
            x_submission.append(x_sub)

            stamps.append([int(df.tail(1).ID),int(df.tail(1).TIME)])


        j+=1
        df2=df2.append(lab.iloc[j:j+1],ignore_index=True)
        df=df.append(vit.iloc[j:j+1], ignore_index=True) # appending each new row to old dataframe and passing to function

x_submission=np.array(x_submission)
print 'Saving x_submission and stamps'
print x_submission.shape
np.save('x_submission.npy',x_submission)
np.save('stamps.npy',stamps)
    

ID = 3595           0
ID = 3596           23
ID = 3597           75
ID = 3598           625
ID = 3599           753
ID = 3600           817
ID = 3601           837
ID = 3602           864
ID = 3603           1107
ID = 3604           1510
ID = 3605           1554
ID = 3606           1826
ID = 3607           1987
ID = 3608           2006
ID = 3609           2254
ID = 3610           2360
ID = 3611           2428
ID = 3612           2498
ID = 3613           2555
ID = 3614           2698
ID = 3615           2892
ID = 3616           2969
ID = 3617           3320
ID = 3618           3598
ID = 3619           3650
ID = 3620           3680
ID = 3621           3725
ID = 3622           3789
ID = 3623           3864
ID = 3624           4013
ID = 3625           4595
ID = 3626           5229
ID = 3627           5395
ID = 3628           5489
ID = 3629           5838
ID = 3630           5961
ID = 3631           6175
ID = 3632           6183
ID = 3633           6290
ID = 3634           6328
ID = 3635   